In [ ]:
import torch
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision import transforms

from origindataset import OriginDataset,Compose
from dataaug import DataAug
from segdataset import SegDataset
from unet import UNet

import matplotlib.pyplot as plt

# Augmentation

In [3]:
# transform_aug = Compose([
#     transforms.RandomHorizontalFlip(p=1),
#     transforms.RandomVerticalFlip(p=1),
#     transforms.Resize((1000, 1000)),
#     transforms.RandomResizedCrop((512, 512)),
#     transforms.ToTensor()
# ])

# original_dataset = OriginDataset('./data/train/image','./data/train/label',transform=transform_aug)
# original_loader = DataLoader(original_dataset, batch_size = 1, shuffle = True, pin_memory = True)

# aug_num = 100
# DataAug(original_loader,aug_num,aug_path='./data/train/aug/')

# Training set loader

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
])

train_dataset = SegDataset('./data/train/aug',transform=transform)
train_loader = DataLoader(train_dataset, batch_size = 2, shuffle = True, pin_memory = True)

# Training

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

unet = UNet().to(device)

unet.train()
unet.to(device)

criterion = nn.BCELoss().cuda()
optimizer = optim.Adam(unet.parameters(),lr=1e-4)


iteration = 0
iterations = []
loss_trains = []
loss_valids = []

correct_valid = 0
total_valid = 0
correct_train = 0
total_train = 0

unet.train()

for epoch in range(10):  # loop over the dataset multiple times
    running_loss_train = 0.0

    for i, data in enumerate(train_loader, 0):
        # get the inputs
        input_train, label_train = data
        del data
        input_train, label_train = input_train.to(device).float(), label_train.to(device).float()
        # zero the parameter gradients
        optimizer.zero_grad()
        pred_train = unet(input_train)
        #del input_train
        loss_train = criterion(pred_train, label_train)

        loss_train.backward()
        optimizer.step()
            
        iteration += 1
        # print statistics
        running_loss_train += loss_train.item()

        loss_trains.append(running_loss_train/iteration)

        iterations.append(iteration)

    print('[%d, %5d] loss_t: %.3f, - iteration : %d' %
          (epoch + 1, i + 1, running_loss_train/iteration, iteration))

    running_loss_train = 0.0
    iteration = 0
    save_path = "./unet_result.pth"
    torch.save(unet.state_dict(), save_path) 
print('Finished Training')

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_trains)